In [1]:
from sqlalchemy import create_engine, text
import pandas as pd 
from datetime import datetime, timedelta

In [2]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)

In [3]:
with engine.connect() as connection:
    connection.execute(text('DROP TABLE IF EXISTS companies'))

2024-04-17 09:31:28,359 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 09:31:28,360 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS companies
2024-04-17 09:31:28,360 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ()
2024-04-17 09:31:28,366 INFO sqlalchemy.engine.Engine ROLLBACK


## Companies 

In [4]:
create_company_table = text('''CREATE TABLE IF NOT EXISTS companies (
    company_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER UNIQUE NOT NULL,
    name VARCHAR(255) NOT NULL,
    industry VARCHAR(255),
    address TEXT,
    contact_email VARCHAR(255),
    website URL,
    description TEXT,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);''')

In [5]:
# Execute SQL statements
with engine.connect() as connection:
    connection.execute(create_company_table)

2024-04-17 09:31:36,532 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 09:31:36,533 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS companies (
    company_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER UNIQUE NOT NULL,
    name VARCHAR(255) NOT NULL,
    industry VARCHAR(255),
    address TEXT,
    contact_email VARCHAR(255),
    website URL,
    description TEXT,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);
2024-04-17 09:31:36,534 INFO sqlalchemy.engine.Engine [generated in 0.00140s] ()
2024-04-17 09:31:36,539 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
pd.read_sql('SELECT * FROM companies',engine)

2024-04-17 09:31:46,957 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 09:31:46,958 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM companies")
2024-04-17 09:31:46,958 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 09:31:46,958 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM companies")
2024-04-17 09:31:46,959 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 09:31:46,959 INFO sqlalchemy.engine.Engine SELECT * FROM companies
2024-04-17 09:31:46,960 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 09:31:46,961 INFO sqlalchemy.engine.Engine ROLLBACK


,company_id,user_id,name,industry,address,contact_email,website,description


In [63]:
def add_company(user_id, name, industry, address, contact_email, website, description):
    # SQL query to insert a new company
    insert_query = text("""
        INSERT INTO companies (user_id, name, industry, address, contact_email, website, description)
        VALUES (:user_id, :name, :industry, :address, :contact_email, :website, :description);
    """)
    
    # Execute the query within a transaction
    with engine.begin() as connection:
        connection.execute(insert_query, {
            "user_id": user_id,
            "name": name,
            "industry": industry,
            "address": address,
            "contact_email": contact_email,
            "website": website,
            "description": description
        })
        print(f"Company {name} added successfully.")


def delete_company(company_id):
    delete_query = text("DELETE FROM companies WHERE company_id = :company_id")

    with engine.begin() as connection:
        result = connection.execute(delete_query, {'company_id': company_id})
        if result.rowcount > 0:
            print(f"Company with ID {company_id} deleted successfully.")
        else:
            print(f"No company found with ID {company_id}, nothing deleted.")


def modify_company(company_id, **kwargs):
    updates = [f"{key} = :{key}" for key in kwargs]
    update_query = text(f"UPDATE companies SET {', '.join(updates)} WHERE company_id = :company_id")
    print(update_query)
    kwargs['company_id'] = company_id  # Adding company_id to the parameters for the WHERE clause

    with engine.begin() as connection:
        result = connection.execute(update_query, kwargs)
        if result.rowcount > 0:
            print(f"Company with ID {company_id} updated successfully.")
        else:
            print(f"No company found with ID {company_id}, nothing updated.")



def get_company(**kwargs):
    if not kwargs:
        print("No parameters provided to search for company.")
        return None
    
    # Constructing the WHERE clause based on kwargs
    conditions = [f"{key} = :{key}" for key in kwargs]
    where_clause = " AND ".join(conditions)
    
    select_query = text(f"SELECT * FROM companies WHERE {where_clause}")

    with engine.connect() as connection:
        result = connection.execute(select_query, kwargs).fetchone()
        if result:
            company_dict = {
                'company_id': result[0],
                'user_id': result[1],
                'name': result[2],
                'industry': result[3],
                'address': result[4],
                'contact_email': result[5],
                'website': result[6],
                'description': result[7]
            }
            return company_dict
        else:
            print("Company not found.")
            return None

In [74]:
pd.read_sql('SELECT * FROM companies',engine)

2024-04-17 10:23:20,391 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 10:23:20,391 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM companies")
2024-04-17 10:23:20,392 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 10:23:20,393 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM companies")
2024-04-17 10:23:20,393 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 10:23:20,394 INFO sqlalchemy.engine.Engine SELECT * FROM companies
2024-04-17 10:23:20,394 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 10:23:20,396 INFO sqlalchemy.engine.Engine ROLLBACK


,company_id,user_id,name,industry,address,contact_email,website,description
0,1,1,dodo,Technology,"123 Tech Drive, Innovation City",contact@techinnovations.com,http://www.techinnovations.com,A leading company in tech innovations aiming t...
1,2,10,Tech Innovations Inc.,Technology,"123 Tech Drive, Innovation City",contact@techinnovations.com,http://www.techinnovations.com,A leading company in tech innovations aiming t...
2,4,1000,Tech Innovations Inc.,Technology,"123 Tech Drive, Innovation City",contact@techinnovations.com,http://www.techinnovations.com,A leading company in tech innovations aiming t...
3,5,1800,Tech Innovations Inc.,Technology,"123 Tech Drive, Innovation City",contact@techinnovations.com,http://www.techinnovations.com,A leading company in tech innovations aiming t...
4,6,1500,Tech Innovations Inc.,Technology,"123 Tech Drive, Innovation City",contact@techinnovations.com,http://www.techinnovations.com,A leading company in tech innovations aiming t...
5,7,1514,Tech Innovations Inc.,Technology,"123 Tech Drive, Innovation City",contact@techinnovations.com,http://www.techinnovations.com,A leading company in tech innovations aiming t...


In [67]:
# Example usage
add_company(
    user_id='1514', 
    name='Tech Innovations Inc.', 
    industry='Technology', 
    address='123 Tech Drive, Innovation City', 
    contact_email='contact@techinnovations.com', 
    website='http://www.techinnovations.com', 
    description='A leading company in tech innovations aiming to...'
)

2024-04-17 10:12:28,558 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 10:12:28,559 INFO sqlalchemy.engine.Engine 
        INSERT INTO companies (user_id, name, industry, address, contact_email, website, description)
        VALUES (?, ?, ?, ?, ?, ?, ?);
    
2024-04-17 10:12:28,560 INFO sqlalchemy.engine.Engine [cached since 2368s ago] ('1514', 'Tech Innovations Inc.', 'Technology', '123 Tech Drive, Innovation City', 'contact@techinnovations.com', 'http://www.techinnovations.com', 'A leading company in tech innovations aiming to...')
Company Tech Innovations Inc. added successfully.
2024-04-17 10:12:28,561 INFO sqlalchemy.engine.Engine COMMIT


In [65]:
modify_company(1,name="dodo")

UPDATE companies SET name = :name WHERE company_id = :company_id
2024-04-17 10:12:06,241 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 10:12:06,241 INFO sqlalchemy.engine.Engine UPDATE companies SET name = ? WHERE company_id = ?
2024-04-17 10:12:06,241 INFO sqlalchemy.engine.Engine [cached since 1570s ago] ('dodo', 1)
Company with ID 1 updated successfully.
2024-04-17 10:12:06,243 INFO sqlalchemy.engine.Engine COMMIT


In [75]:
get_company(user_id = 1000)

2024-04-17 10:23:28,348 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 10:23:28,348 INFO sqlalchemy.engine.Engine SELECT * FROM companies WHERE user_id = ?
2024-04-17 10:23:28,349 INFO sqlalchemy.engine.Engine [cached since 1408s ago] (1000,)
2024-04-17 10:23:28,350 INFO sqlalchemy.engine.Engine ROLLBACK


{'company_id': 4,
 'user_id': 1000,
 'name': 'Tech Innovations Inc.',
 'industry': 'Technology',
 'address': '123 Tech Drive, Innovation City',
 'contact_email': 'contact@techinnovations.com',
 'website': 'http://www.techinnovations.com',
 'description': 'A leading company in tech innovations aiming to...'}

In [72]:
delete_company(3)

2024-04-17 10:22:20,629 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 10:22:20,629 INFO sqlalchemy.engine.Engine DELETE FROM companies WHERE company_id = ?
2024-04-17 10:22:20,630 INFO sqlalchemy.engine.Engine [generated in 0.00049s] (3,)
Company with ID 3 deleted successfully.
2024-04-17 10:22:20,632 INFO sqlalchemy.engine.Engine COMMIT
